In [32]:
import torch

import numpy as np
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torchvision.transforms import Compose 
import matplotlib.pyplot as plt


In [33]:
!pip install PyDrive
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
from google.colab import drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [34]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [35]:
downloaded = drive.CreateFile({'id':"15NU7LovMkq_4OTjwz8AD5fvCD0zdVDyi"})
downloaded.GetContentFile('project1_modelkj.pt') 
# !unzip '/content/DataSet.zip' -d '/content/files' 

In [36]:
class BasicBlock(nn.Module):   
      def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
  
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

      def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out



class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.linear = nn.Linear(256, num_classes)


    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def project1_model():
    return ResNet(BasicBlock, [5,4,3])

In [37]:
model = project1_model().cuda()

In [38]:
import torch

import numpy as np
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torchvision.transforms import Compose 
import matplotlib.pyplot as plt
model = project1_model()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = project1_model().to(device)
model.load_state_dict(torch.load('./project1_modelkj.pt'), strict=False)


<All keys matched successfully>

In [39]:
from torchvision.transforms import transforms
test_transform = Compose([
    transforms.ToTensor(), #converting all input test images to tensors
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2470, 0.2435, 0.2616]) #normalizing tensor values
])
testdata = torchvision.datasets.CIFAR10('./CIFAR10/',train=False,download=True,transform=test_transform)
print("Test size: ",len(testdata)) #printing the test data size

Files already downloaded and verified
Test size:  10000


In [40]:
testDataLoader = torch.utils.data.DataLoader(testdata,batch_size=64,shuffle=False)

In [41]:
from torch.optim import optimizer
loss = torch.nn.CrossEntropyLoss()

In [42]:

test_loss_hist = [] 
test_acc_history = [] 
test_loss = 0.0 
correct_points_test = 0 
for i,data in enumerate(testDataLoader):
  with torch.no_grad():
    images, labels = data
    images = images.cuda()
    labels = labels.cuda()

    y_pred = model(images).cuda()
    fit = loss(y_pred,labels)
    test_loss += fit.item()
    pred = y_pred.argmax(dim=1, keepdim=True) # get the index of the max log-probability
    correct_points_test += pred.eq(labels.view_as(pred)).sum().item()
test_loss = test_loss/len(testDataLoader)
print("test_loss = ",test_loss)
test_acc = correct_points_test/len(testDataLoader.dataset)
print("test_accuracy = ",test_acc)
test_loss_hist.append(test_loss) 
test_acc_history.append(test_acc)

test_loss =  0.3796435270138132
test_accuracy =  0.9311


In [43]:
test_acc_history

[0.9311]